In [ ]:
##!/usr/bin/env python
"""plot_transect.py

Script plots the HGT vs Longitude cross section as the difference between the two nested domains:
d02: 800m resolution with urban LCZs
d03: 800m resolution where all urban areas are replaced with grass

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Tue Sep  1 14:37:33 AEST 2020

"""

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Load Packages

In [ ]:
#from __future__ import division
import numpy as np
import pandas as pd
import math
import netCDF4 as nc
import sys
import os
import glob as glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
from matplotlib import cm
from matplotlib.collections import LineCollection
import common_functions as cf
import datetime as dt
import wrf
from scipy import stats
import metpy.calc as mpcalc

### Experimental Details of the Model Data

In [ ]:
# Simulation Period
syear = 2017
smon = 1
sday = 2 
eyear = 2017
emon = 2
eday = 28  # Add an extra day so that the 27th Feb data is included
simlen = dt.datetime(eyear,emon,eday) - dt.datetime(syear,smon,sday)
nst = (simlen.days * 24 * 6) # No. simulations days x 24 hours in a day x 6 history intervals per hour

# Dates - Used for subsetting the AWS data so you pick the day before the start date and the day after the end date
sdate = "2017-01-01"
edate = "2017-02-28"

# Data directory 
datadir='/g/data/w97/azh561/WRF/'
ensmem = ['sydney800m','sydney800m','sydney800m_06H','sydney800m_06H','sydney800m_12H','sydney800m_12H','sydney800m_18H','sydney800m_18H','sydney800m_00H','sydney800m_00H'] 
rlabels = ['U1','G1','U2','G2','U3','G3','U4','G4','U5','G5']
domain = ["d02","d03","d02","d03","d02","d03","d02","d03","d02","d03"]
nmem = len(ensmem)

# Landsea mask
mask_file='/g/data/w97/azh561/WRF/sydney800m/geo_em.%s.nc' %(domain[0])
f = nc.Dataset(mask_file)
lu = f.variables['LU_INDEX'][0,:,:]
luf = f.variables['LANDUSEF'][0,:,:,:]
lat2d = f.variables['XLAT_M'][0,:,:]
lontmp = f.variables['XLONG_M'][0,:,:]
lon2d = np.where(lontmp<0.0,lontmp+360,lontmp)
hgt2d = f.variables['HGT_M'][0,:,:]
lsmask = f.variables['LANDMASK'][0,:,:]
clon = f.getncattr('CEN_LON')
nlu = f.getncattr('NUM_LAND_CAT')
iswater = f.getncattr('ISWATER')
nlat,nlon = lon2d.shape
f.close()

nlev = 44

# LCZs
LCZnm = ['Compact high-rise','Compact midrise','Compact low-rise','Open high-rise',
         'Open low-rise','Lightweight low-rise','Large low-rise','Sparsely built','Heavy industry']

# Figure Details
fig_dir='%s/figures/' %(os.getcwd())
fig_name_prefix='LCZ_'
if not os.path.exists(fig_dir):
  os.makedirs(fig_dir)


In [ ]:
start = dt.datetime(syear,smon,sday,0,0,0)
end = dt.datetime(eyear,emon,eday,0,0,0)
days = (end - start).days
ntim = days * 24 * 60
datelist = [start + dt.timedelta(minutes=x) for x in range(ntim+1)]
# Get the day-month hour-minutes on 10 minute interval
ftimes = np.asarray([datelist[x].strftime("%m-%d %H-%M") for x in range(ntim+1)])[::10]
fdates = np.asarray([datelist[x].strftime("%m-%d") for x in range(ntim+1)])[::10]
fhours = np.asarray([datelist[x].strftime("%H") for x in range(ntim+1)])[::10]
fdays = np.asarray([datelist[x].strftime("%m-%d") for x in range(ntim+1)])[::10]

### Define the East-West Transect to look at

In [ ]:
i0 = 174 # For d02. If d01 use 114
hgti0 = hgt2d[i0,:]

In [ ]:
luma = [None] * len(lu[i0,:])
for ll in range(len(luma)):
    if lu[i0,ll] in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]:
        luma[ll] = 'l'
    elif lu[i0,ll] == 17:
        luma[ll] = 'o'
    else:
        luma[ll] = 'u'

### Split analysis by heatwave periods

From the Bureau of Meteorology Special Climate Statement 61 there were 3 heatwaves:

    10-14 January
    17-21 January
    31 January - 12 February 
    
For the latter heatwave this was terminated by a cold front.

So here we examine separately the week before the first heatwave, each heatwave period and the week after the third heatwave

If using the Sydney Airport Data as well as Richmond and Observatory Hill (via Scorcher) the dates for the heatwaves are:

    11-14 January
    16-18 January
    30 January - 1 February
    5-7 February
    10-12 February

In [ ]:
HW1S = [i for i in range(len(ftimes)) if ftimes[i] in ['01-11 00-00']][0]
HW1F = [i for i in range(len(ftimes)) if ftimes[i] in ['01-14 12-00']][0]

HW2S = [i for i in range(len(ftimes)) if ftimes[i] in ['01-16 00-00']][0]
HW2F = [i for i in range(len(ftimes)) if ftimes[i] in ['01-18 12-00']][0]

HW3S = [i for i in range(len(ftimes)) if ftimes[i] in ['01-30 00-00']][0]
HW3F = [i for i in range(len(ftimes)) if ftimes[i] in ['02-01 12-00']][0]

HW4S = [i for i in range(len(ftimes)) if ftimes[i] in ['02-05 00-00']][0]
HW4F = [i for i in range(len(ftimes)) if ftimes[i] in ['02-07 12-00']][0]

HW5S = [i for i in range(len(ftimes)) if ftimes[i] in ['02-10 00-00']][0]
HW5F = [i for i in range(len(ftimes)) if ftimes[i] in ['02-12 12-00']][0]


### Extract model data corresponding to an East-West Transect through the city (latitude index of 174)

- run this on gadi to avoid memory limits

In [ ]:
for mm in range(nmem):
    
    tk = np.empty((len(ftimes)-1,nlev,nlon),dtype=np.float64)
    qv = np.empty((len(ftimes)-1,nlev,nlon),dtype=np.float64)
    w = np.empty((len(ftimes)-1,nlev,nlon),dtype=np.float64)
    
    # Files list
    filelist = sorted(glob.glob('%s/%s/WRF_output/%s/wrfout_%s_2017-*' %(datadir,ensmem[mm],domain[mm],domain[mm])))
    nfile = len(filelist)
    for ff in range(int(nfile/24)):

        wrffiles = [nc.Dataset(filelist[(ff*24)]),nc.Dataset(filelist[(ff*24)+1]),nc.Dataset(filelist[(ff*24)+2])
        ,nc.Dataset(filelist[(ff*24)+3]),nc.Dataset(filelist[(ff*24)+4]),nc.Dataset(filelist[(ff*24)+5])
        ,nc.Dataset(filelist[(ff*24)+6]),nc.Dataset(filelist[(ff*24)+7]),nc.Dataset(filelist[(ff*24)+8])
        ,nc.Dataset(filelist[(ff*24)+9]),nc.Dataset(filelist[(ff*24)+10]),nc.Dataset(filelist[(ff*24)+11])
        ,nc.Dataset(filelist[(ff*24)+12]),nc.Dataset(filelist[(ff*24)+13]),nc.Dataset(filelist[(ff*24)+14])
        ,nc.Dataset(filelist[(ff*24)+15]),nc.Dataset(filelist[(ff*24)+16]),nc.Dataset(filelist[(ff*24)+17])
        ,nc.Dataset(filelist[(ff*24)+18]),nc.Dataset(filelist[(ff*24)+19]),nc.Dataset(filelist[(ff*24)+20])
        ,nc.Dataset(filelist[(ff*24)+21]),nc.Dataset(filelist[(ff*24)+22]),nc.Dataset(filelist[(ff*24)+23])]

        tk[144*ff:144*(ff+1),:,:] = wrf.getvar(wrffiles,"tk",timeidx=None,method='cat')[:144,:,i0,:]
        qv[144*ff:144*(ff+1),:,:] = wrf.getvar(wrffiles,"QVAPOR",timeidx=None,method='cat')[:144,:,i0,:]
        w[144*ff:144*(ff+1),:,:] = wrf.getvar(wrffiles,"wa",timeidx=None,method='cat')[:144,:,i0,:]
         
        if ff == 0:
            z = wrf.getvar(wrffiles,"z",timeidx=None,method='cat')[0,:,0,0]

        for a in range(24):
            wrffiles[a].close()
        
    del filelist,nfile
    
    # Write output to file
        
    dataset = nc.Dataset('/g/data/w97/azh561/WRF/processed/wrfout_East-West_Transect_%s_%s.nc' %(ensmem[mm],domain[mm]),'w') # open file

    # Create dimensions
    time = dataset.createDimension('time',(len(ftimes)-1))
    lev = dataset.createDimension('lev',nlev)
    lon = dataset.createDimension('lon',nlon)

    # Create coordinate variables
    times = dataset.createVariable('time',ftimes.dtype,('time',))
    levels = dataset.createVariable('lev',np.float64,('lev',))
    longitude = dataset.createVariable('lon',np.float64,('lon',))

    # Create variables
    T = dataset.createVariable('T', np.float64,('time','lev','lon'))
    Q = dataset.createVariable('Q', np.float64,('time','lev','lon'))
    W = dataset.createVariable('W', np.float64,('time','lev','lon'))
    Z = dataset.createVariable('Z', np.float64,('lev'))

    # Write data
    T[:] = tk[:] 
    Q[:] = qv[:] 
    W[:] = w[:] 
    Z[:] = z[:] 
    times[:] = ftimes[:-1]
    levels[:] = np.arange(1,nlev+1)
    longitude[:] = lon2d[i0,:]

    # Write the file
    dataset.close()
    
    del tk,qv,w,z

### Read in the extracted data and plot
- calculate the ensemble average on Gadi using ncea

In [ ]:
# Get urban
file = nc.Dataset('/g/data/w97/azh561/WRF/processed/wrfout_East-West_Transect_%s.nc' %('d02'),'r')
Tu = file.variables['T'][:,:,:] - 273.15 # convert to degC
Qu = file.variables['Q'][:,:,:]
Wu = file.variables['W'][:,:,:]
Uu = file.variables['U'][:,:,:]
Vu = file.variables['V'][:,:,:]
Z = file.variables['Z'][:]
file.close()

# Get grass
file = nc.Dataset('/g/data/w97/azh561/WRF/processed/wrfout_East-West_Transect_%s.nc' %('d03'),'r')
Tg = file.variables['T'][:,:,:] - 273.15 # convert to degC
Qg = file.variables['Q'][:,:,:]
Wg = file.variables['W'][:,:,:]
Ug = file.variables['U'][:,:,:]
Vg = file.variables['V'][:,:,:]
file.close()


### Plot time series to examine sea breeze and Foehn wind effects

In [ ]:
# Function to plot data
def plot_ts(time,tsdata,tind,vlabels,llabel,figurename,lspace):

    """This function plots time series for observations and models"""

    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    import string
    import scipy
    
    # Figure formatting
    plt.rcParams['savefig.dpi']=300
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    plt.rcParams["font.size"] = 18


    # Define dimensions
    nvar = tsdata.shape[0]
    nt = tsdata.shape[1]
    nmod = tsdata.shape[2]
      
    # Create figure object and subplots
    fig, ax = plt.subplots(nvar, 1, figsize=(30.0,5.0*(nvar)), squeeze=False)
    tarr = np.arange(0,nt)
    evenly_spaced_interval = np.linspace(0, 1, nmod)
    mycolors = [plt.cm.coolwarm(x) for x in evenly_spaced_interval]
    
    # Iterate through variables
    for vind in range(nvar):

        for mm in range(nmod):
            ax[vind,0].plot(tarr,tsdata[vind,:,mm], linewidth=1,color=mycolors[mm], linestyle='-',label=llabel[mm])
       
        # Fix Labelling
        ax[vind,0].set_ylabel('%s' %(vlabels[vind]), fontweight = 'bold',fontsize=18)
        ax[vind,0].set_title('(%s)' %(string.ascii_lowercase[vind]), fontweight='bold', fontsize=18, y = 0.9, x = 0.015)
 
        # Amend axis limits
        ax[vind,0].set_xlim(tarr[0],tarr[-1])
              
        if vind < nvar-1:
            ax[vind,0].set_xticks([],[])
        else:
            ax[vind,0].set_xticks(tarr[::lspace])
            ax[vind,0].set_xticklabels(time[::lspace],rotation=90,fontsize=18)
    
        # Add vertical line at dates of interest
        #for ll in range(len(tind)): 
        #    ax[vind,0].axvline(tind[ll], color='grey', linestyle='--',linewidth=3.0)
        ax[vind,0].axvspan(tind[0], tind[1], alpha=0.25, color='red')
        ax[vind,0].axvspan(tind[2], tind[3], alpha=0.25, color='red')
        ax[vind,0].axvspan(tind[4], tind[5], alpha=0.25, color='red')
        ax[vind,0].axvspan(tind[6], tind[7], alpha=0.25, color='red')
        ax[vind,0].axvspan(tind[8], tind[9], alpha=0.25, color='red')

    # Add text to label the heatwaves above the first plot
    ax[0,0].text(0.1875,1.05,'%s HW' %('$1^{st}$'), 
                            horizontalalignment='center',verticalalignment='center',transform = ax[0,0].transAxes,
                            color='black', fontweight='bold', fontsize=18)
    ax[0,0].text(0.265,1.05,'%s HW' %('$2^{nd}$'), 
                            horizontalalignment='center',verticalalignment='center',transform = ax[0,0].transAxes,
                            color='black', fontweight='bold', fontsize=18)
    ax[0,0].text(0.515,1.05,'%s HW' %('$3^{rd}$'), 
                            horizontalalignment='center',verticalalignment='center',transform = ax[0,0].transAxes,
                            color='black', fontweight='bold', fontsize=18)
    ax[0,0].text(0.62,1.05,'%s HW' %('$4^{th}$'), 
                            horizontalalignment='center',verticalalignment='center',transform = ax[0,0].transAxes,
                            color='black', fontweight='bold', fontsize=18)
    ax[0,0].text(0.705,1.05,'%s HW' %('$5^{th}$'), 
                            horizontalalignment='center',verticalalignment='center',transform = ax[0,0].transAxes,
                            color='black', fontweight='bold', fontsize=18)
                 
    ax[-2,0].axhline(0., color='grey', linestyle='--',linewidth=3.0)
    ax[-1,0].axhline(0., color='grey', linestyle='--',linewidth=3.0)

    
    fig.tight_layout()
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.savefig(figurename,bbox_inches='tight')
#    plt.close(fig)


In [ ]:
tind = [HW1S,HW1F,HW2S,HW2F,HW3S,HW3F,HW4S,HW4F,HW5S,HW5F]
vlabels = ['$T$ [\xb0 C]','Q [kg $kg^{-1}$]','$U$ [m $s^{-1}$]','$V$ [m $s^{-1}$]']
lspace = 144 # As the wrf output is saved at a 10 minute interval
figurename = 'Urban_Lons_Times_Series_SB_Foehn.png'

# Aggregate
tsdata = np.empty((4,len(ftimes)-1,45),dtype=np.float64)
tsdata[0,:,:] = Tu[:,0,240:285]
tsdata[1,:,:] = Qu[:,0,240:285]
tsdata[2,:,:] = Uu[:,0,240:285]
tsdata[3,:,:] = Vu[:,0,240:285]


plot_ts(fdates[:-1],tsdata[:,:,::-1],tind,vlabels,lon2d[i0,240:285],figurename,lspace)

Calculate the temperature drop and increase over different intervals

In [ ]:
# Define function to smooth the timeseries
# Taken from http://scipy-cookbook.readthedocs.io/items/SignalSmooth.html
def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """
    
    if x.ndim != 1:
        sys.exit("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        sys.ext("Input vector needs to be bigger than window size.")

    if window_len<3:
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        sys.exit("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")

    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y


In [ ]:
timeinterval = 6*24
tschanges = np.empty((2,len(ftimes)-timeinterval,45),dtype=np.float64)
for tt in range(len(ftimes[:-timeinterval])-1):
    tschanges[:,tt,:] = tsdata[:,tt,:] - tsdata[:,tt+timeinterval,:]



In [ ]:
tind = [HW1S,HW1F,HW2S,HW2F,HW3S,HW3F,HW4S,HW4F,HW5S,HW5F]
vlabels = ['$T$ [\xb0 C]','$U$ [m $s^{-1}$]']
lspace = 144 # As the wrf output is saved at a 10 minute interval
figurename = 'Smoothed_Urban_Lons_Times_Series_Changes_over_24hr.png'

plot_ts(ftimes[:-((6*24)+1)],tschanges[:,:-1,::-1],tind,vlabels,lon2d[i0,240:285],figurename,lspace)


Calculate the urban gradient across the city

In [ ]:
[np.nanmin(Tu[:,0,240]-Tu[:,0,284]),np.nanmean(Tu[:,0,240]-Tu[:,0,284]),np.nanmedian(Tu[:,0,240]-Tu[:,0,284]),np.nanmax(Tu[:,0,240]-Tu[:,0,284]) ]

As above but plot the difference between the urban and grass realisations

In [ ]:
tind = [HW1S,HW1F,HW2S,HW2F,HW3S,HW3F,HW4S,HW4F,HW5S,HW5F]
vlabels = ['$T$ [\xb0 C]','Q [kg $kg^{-1}$]','$U$ [m $s^{-1}$]','$V$ [m $s^{-1}$]']
lspace = 144 # As the wrf output is saved at a 10 minute interval
figurename = 'Urban_minus_Grass_Lons_Times_Series_SB_Foehn.png'

# Aggregate
tsdiff = np.empty((4,len(ftimes)-1,45),dtype=np.float64)
tsdiff[0,:,:] = Tu[:,0,240:285] - Tg[:,0,240:285]
tsdiff[1,:,:] = Qu[:,0,240:285] - Qg[:,0,240:285]
tsdiff[2,:,:] = Uu[:,0,240:285] - Ug[:,0,240:285]
tsdiff[3,:,:] = Vu[:,0,240:285] - Vg[:,0,240:285]


plot_ts(fdates[:-1],tsdiff[:,:,::-1],tind,vlabels,lon2d[i0,240:285],figurename,lspace)

Zoom in on the heatwave periods

In [ ]:
# Function to plot data
def plot_ts(time,tsdata,tind,vlabels,llabel,figurename,lspace):

    """This function plots time series for observations and models"""

    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    import string
    import scipy
    
    # Figure formatting
    plt.rcParams['savefig.dpi']=300
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"

    # Define dimensions
    nvar = tsdata.shape[0]
    nt = tsdata.shape[1]
    nmod = tsdata.shape[2]
      
    # Create figure object and subplots
    fig, ax = plt.subplots(nvar, 1, figsize=(30.0,5.0*(nvar)), squeeze=False)
    tarr = np.arange(0,nt)
    evenly_spaced_interval = np.linspace(0, 1, nmod)
    mycolors = [plt.cm.coolwarm(x) for x in evenly_spaced_interval]
    
    # Iterate through variables
    for vind in range(nvar):

        for mm in range(nmod):
            ax[vind,0].plot(tarr,tsdata[vind,:,mm], linewidth=1,color=mycolors[mm], linestyle='-',label=llabel[mm])
       
        # Fix Labelling
        ax[vind,0].set_ylabel('%s' %(vlabels[vind]), fontweight = 'bold',fontsize=18)
        ax[vind,0].set_title('(%s)' %(string.ascii_lowercase[vind]), fontweight='bold', fontsize=18, y = 0.9, x = 0.015)
 
        # Amend axis limits
        ax[vind,0].set_xlim(tarr[0],tarr[-1])
        
        if vind < nvar-1:
            ax[vind,0].set_xticks([],[])
        else:
            ax[vind,0].set_xticks(tarr[::lspace])
            ax[vind,0].set_xticklabels(time[::lspace],rotation=90,fontsize=18)
                 
    ax[-2,0].axhline(0., color='grey', linestyle='--',linewidth=3.0)
    ax[-1,0].axhline(0., color='grey', linestyle='--',linewidth=3.0)

    
#    legend = ax[-1,0].legend(loc='upper center', bbox_to_anchor=(0.5,-0.4), ncol=6, fontsize=14)    
    fig.tight_layout()
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.savefig(figurename,bbox_inches='tight')
#    fig.savefig(figurename, bbox_extra_artists=(legend,),bbox_inches='tight')
#    plt.close(fig)


In [ ]:
tind = [HW1S,HW1F,HW2S,HW2F,HW3S,HW3F,HW4S,HW4F,HW5S,HW5F]
sind = [HW1S,HW2S,HW3S,HW4S,HW5S]
find = [HW1F,HW2F,HW3F,HW4F,HW5F]
hwlab = ["HW1","HW2","HW3","HW4","HW5"]
vlabels = ['$T$ [\xb0 C]','Q [kg $kg^{-1}$]','$U$ [m $s^{-1}$]','$V$ [m $s^{-1}$]']
lspace = 144 # As the wrf output is saved at a 10 minute interval

for hh in range(5):

    hwtime = ftimes[(sind[hh]-(3*144)):find[hh]]
    
    # Plot urban
    figurename = 'Urban_Lons_Times_Series_%s.png' %(hwlab[hh])
    tsdata = np.empty((4,len(hwtime),45),dtype=np.float64)
    tsdata[0,:,:] = Tu[(sind[hh]-(3*144)):find[hh],0,240:285]
    tsdata[1,:,:] = Qu[(sind[hh]-(3*144)):find[hh],0,240:285]
    tsdata[2,:,:] = Uu[(sind[hh]-(3*144)):find[hh],0,240:285]
    tsdata[3,:,:] = Vu[(sind[hh]-(3*144)):find[hh],0,240:285]
    plot_ts(hwtime,tsdata[:,:,::-1],tind,vlabels,lon2d[i0,240:285],figurename,lspace)
    
    # Plot grass
    figurename = 'Grass_Lons_Times_Series_%s.png' %(hwlab[hh])
    tsgras = np.empty((4,len(hwtime),45),dtype=np.float64)
    tsgras[0,:,:] = Tg[(sind[hh]-(3*144)):find[hh],0,240:285]
    tsgras[1,:,:] = Qg[(sind[hh]-(3*144)):find[hh],0,240:285]
    tsgras[2,:,:] = Ug[(sind[hh]-(3*144)):find[hh],0,240:285]
    tsgras[3,:,:] = Vg[(sind[hh]-(3*144)):find[hh],0,240:285]
    plot_ts(hwtime,tsgras[:,:,::-1],tind,vlabels,lon2d[i0,240:285],figurename,lspace)
    
    
    # Plot urban minus grass
    figurename = 'Urban_minus_Grass_Lons_Times_Series_%s.png' %(hwlab[hh])
    tsdiff = np.empty((4,len(hwtime),45),dtype=np.float64)
    tsdiff[0,:,:] = Tu[(sind[hh]-(3*144)):find[hh],0,240:285] - Tg[(sind[hh]-(3*144)):find[hh],0,240:285]
    tsdiff[1,:,:] = Qu[(sind[hh]-(3*144)):find[hh],0,240:285] - Qg[(sind[hh]-(3*144)):find[hh],0,240:285]
    tsdiff[2,:,:] = Uu[(sind[hh]-(3*144)):find[hh],0,240:285] - Ug[(sind[hh]-(3*144)):find[hh],0,240:285]
    tsdiff[3,:,:] = Vu[(sind[hh]-(3*144)):find[hh],0,240:285] - Vg[(sind[hh]-(3*144)):find[hh],0,240:285]
    plot_ts(hwtime,tsdiff[:,:,::-1],tind,vlabels,lon2d[i0,240:285],figurename,lspace)
    
    del tsdata,tsgras,tsdiff,hwtime

### Urban vs Grass Contrast for paper 

Using the whole simulation period as the contrast seems fairly consistent between the events

In [ ]:
def plot_contrast(Tu,Tg,Qu,Qg,Z,lon1d,hgt,luma,tlabel,figurename,mx,mn,mxmn):

    import matplotlib as mpl
    import matplotlib.pyplot as plt
    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    from matplotlib import colors
    from matplotlib import cm
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes
    import cartopy.crs as ccrs

    # Figure formatting
    plt.rcParams['savefig.dpi']=500
#    plt.rcParams['savefig.dpi']=100
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    
    nrow = 2
    ncol = 2
    nbins = 20
    wratios = np.repeat(1.,ncol)
    wratios[-1] = 0.05   
    gs = mpl.gridspec.GridSpec(nrows=nrow,ncols=ncol, width_ratios=wratios, wspace=0.025)
    fig = plt.figure(figsize=(24.0,8.0))
    
    # Make axes
    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[1,0])
    
    # Colour bar axes (':' as the colour bars cover multiple rows)
    # Use a new subplot so we can control the spacing better
    cgs1 = mpl.gridspec.GridSpecFromSubplotSpec(nrows=nrow,ncols=1, subplot_spec=gs[:,1], wspace=0.1)
    cax1 = plt.subplot(cgs1[0,0])
    cax2 = plt.subplot(cgs1[1,0])


    # Convert lon1d and Z into 2D arrays for including topography
    lons = np.repeat(lon1d[np.newaxis],len(Z),axis=0)
    Z2d = np.empty(lons.shape,np.float64)
    for ll in range(len(lon1d)):
        for zz in range(len(Z)):
            if zz == 0:
                Z2d[0,ll] = hgt[ll]# Z[zz] - hgt[ll]
            else:
                Z2d[zz,ll] = Z[zz] + hgt[ll]
      
    # Mask small values
    #np.ma.masked_inside((Tu-Tg), -mxmn[0]/10., mxmn[0]/10.)
    
    # Define the levels for the contour lines
    
    ctlevels = MaxNLocator(nbins=nbins).tick_values(-mxmn[0],mxmn[0])
    ntnorm = BoundaryNorm(ctlevels,ncolors=plt.get_cmap('seismic').N, clip=True)
    
    cqlevels = MaxNLocator(nbins=nbins).tick_values(-mxmn[1],mxmn[1])
    nqnorm = BoundaryNorm(cqlevels,ncolors=plt.get_cmap('BrBG').N, clip=True)
    
    # Temperature
    cm1 = ax1.contourf(lons,Z2d,(Tu-Tg), vmin=-mxmn[0],vmax=mxmn[0],cmap='seismic', levels=ctlevels, norm=ntnorm, extend='both')
    ax1.set_ylabel('Height [km]', fontweight = 'bold',fontsize=14)
#    ax1.set_title('(a) T [\xb0 C] Urban - Grass', fontweight='bold',loc='left')
    ax1.set_title('(a) T [\xb0 C]', fontweight='bold',loc='left')
    
    # Moisture
    cm2 = ax2.contourf(lons,Z2d,(Qu-Qg), vmin=-mxmn[1],vmax=mxmn[1],cmap='BrBG', levels=cqlevels, norm=nqnorm, extend='both')
    ax2.set_ylabel('Height [km]', fontweight = 'bold',fontsize=14)
#    ax2.set_title('(b) Q [kg $kg^{-1}$] Urban - Grass', fontweight='bold',loc='left')
    ax2.set_title('(b) Q [kg $kg^{-1}$]', fontweight='bold',loc='left')
    
    # Add sillouette of topography - to be checked!
    ax1.fill_between(lon1d, hgt, 0, interpolate=True, color='black')
    ax2.fill_between(lon1d, hgt, 0, interpolate=True, color='black')
    
    # Add land use type
    lucol = { 'l' : 'green',
            'o' : 'blue',
            'u' : 'grey'}
    c = [lucol[val] for val in luma]
    ax1.scatter(lon1d,np.zeros(len(lon1d))-0.075,c=c,marker='s')
    ax2.scatter(lon1d,np.zeros(len(lon1d))-0.075,c=c,marker='s')

    # Add date-time
#    ax1.text(0.825,0.90,'%s' %(tlabel), # For averaging across time intervals
#    ax1.text(0.9,0.90,'%s' %(tlabel),  # For individual timesteps
#                            horizontalalignment='center',verticalalignment='center',transform = ax1.transAxes,
#                            fontweight='bold', fontsize=18)
    
    # Amend x-axis
    ax1.set_xticks([],[])
    ax2.set_xticks(lon1d[12::24])
    ax2.set_xticklabels(np.round(lon1d[12::24],2),rotation=90,fontsize=18)

    # Amend y-axis
    ax1.set_ylim(-0.1,5)
    ax2.set_ylim(-0.1,5)
    
    plt.colorbar(cm1, cax1)
    plt.colorbar(cm2, cax2)

    fig.tight_layout()
    fig.subplots_adjust(wspace=0, hspace=0.15)
    plt.savefig(figurename, bbox_inches='tight')
    plt.close(fig)


In [ ]:
mx = [45.0,0.015]
mn = [-45.0,0.0]
mxmn = [1.0,0.0005]

# For different time periods of the simulation
figurename = 'East-West_Transect_urban_vs_grass_whole_period.png'
dtlb = '2017-%s to 2017-%s' %(ftimes[0],ftimes[-1])
plot_contrast(np.nanmean(Tu,axis=0),np.nanmean(Tg,axis=0),
          np.nanmean(Qu,axis=0),np.nanmean(Qg,axis=0),
          Z/1000.,lon2d[i0,:],hgti0/1000.,luma,dtlb,figurename,mx,mn,mxmn)

Cumulative measures across the heatwave periods

In [ ]:
mx = [45.0,0.015]
mn = [-45.0,0.0]
mxmn = [10.0,0.01]

# Calculate the urban minus grass different
Tcontrast = Tu - Tg
Qcontrast = Qu - Qg

# Calculate the sum over the heatwave periods
# Here we calculate the mean difference between urban and non-urban for the day and then
# sum these across the heatwave days
T0 = np.zeros((Tu.shape[1],Tu.shape[2]),dtype=np.float64)
Q0 = np.zeros((Qu.shape[1],Qu.shape[2]),dtype=np.float64)
cumulativeheat = np.zeros((Tu.shape[1],Tu.shape[2]),dtype=np.float64)
cumulativemoist = np.zeros((Qu.shape[1],Qu.shape[2]),dtype=np.float64)

hwdayst = [HW1S,HW1S+144,HW1S+(2*144),HW1S+(3*144), HW2S,HW2S+144,HW2S+(2*144), HW3S,HW3S+144,HW3S+(2*144), HW4S,HW4S+144,HW4S+(2*144), HW5S,HW5S+144,HW5S+(2*144)]
hwdayfi = [HW1S+144,HW1S+(2*144),HW1S+(3*144),HW1F, HW2S+144,HW2S+(2*144),HW2F, HW3S+144,HW3S+(2*144),HW3F, HW4S+144,HW4S+(2*144),HW4F, HW5S+144,HW5S+(2*144),HW5F]

for hh in range(len(hwdayst)):
    tmp = np.nansum(Tcontrast[hwdayst[hh]:hwdayfi[hh],:,:],axis=0) / Tcontrast[hwdayst[hh]:hwdayfi[hh],:,:].shape[0]
    cumulativeheat += tmp
    qmp = np.nansum(Qcontrast[hwdayst[hh]:hwdayfi[hh],:,:],axis=0) / Qcontrast[hwdayst[hh]:hwdayfi[hh],:,:].shape[0]
    cumulativemoist += qmp
    del tmp,qmp
    
# For different time periods of the simulation
figurename = 'East-West_Transect_urban_vs_grass_whole_period_cumulative.png'
dtlb = '2017-%s to 2017-%s' %(ftimes[0],ftimes[-1])
plot_contrast(cumulativeheat,T0,cumulativemoist,Q0,
          Z/1000.,lon2d[i0,:],hgti0/1000.,luma,dtlb,figurename,mx,mn,mxmn)